In [ ]:
%matplotlib inline

In [ ]:
import sys
import numpy as np
import scipy as sp
import scipy.signal
import radarkit
import chart
import matplotlib
import matplotlib.pyplot as plt
import time

In [ ]:
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = ['Arial', 'Helvetica']
matplotlib.rcParams['font.sans-serif'] = ['System Font', 'Verdana', 'Arial']
matplotlib.rcParams['figure.figsize'] = (8.89, 5)   # Change the size of plots
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
sweep = radarkit.read('/Users/boonleng/Downloads/PX-20170220-050706-E2.4-Z.nc')

z = sweep['products']['Z']['data']
d = sweep['products']['D']['data']
p = sweep['products']['P']['data']
r = sweep['products']['R']['data']

In [ ]:
alpha = 0.32        # Typical value found from SCWC
beta = 0.455        # Typical value fround from SCWC
b = 1.02            # The b factor, 1.02 for X-band

## Calibration / Data Extraction

In [ ]:
rayCount = z.shape[0]
gateCount = z.shape[1]
gateSpacing = 1.0e-3 * sweep['gateSizeMeters']
downSamplingRatio = int(sweep['gateSizeMeters'] / 30.0)

# Undo range correction and the ZCal in DSP to estimate the SNR
rr = 1.0e-3 * (np.arange(sweep['gateCount']) + 0.5) * sweep['gateSizeMeters']
snr = z - 20.0 * np.log10(rr) + 23

# Transition gate at 10.5 km at the given spacing
g = int((69.0e-6 * 3.0e8 * 0.5) / 30.0) + 5
g = int((g + downSamplingRatio - 1) / downSamplingRatio)

print(g)

z_off = 10.0 * np.log10(1.5 / 67)
snr[:, :g] = snr[:, :g] + z_off

# PhiDP calibration so that the transition is smooth and PhiDP starts ~ 0 deg
p[:, :g] = p[:, :g] + 40;
p[:, g:] = p[:, g:] + 44;

In [ ]:
plt.figure()

plt.subplot(121)
_ = plt.plot(snr.T)
plt.title('Estimated SNR')

plt.subplot(122)
plt.imshow(snr)
plt.clim(-10, 60)
plt.colorbar(orientation='horizontal')
_ = plt.title('Estimated SNR')

In [ ]:
# Make a copy of PhiDP
pp = np.copy(p)

# Set PhiDP < 0 to 0 (no correction)
pp[np.nan_to_num(pp) < 0.0] = 0.0

# Attenuation
az = alpha * pp ** b
ad = beta / alpha * az

# Correction
zc = z + az
dc = d + ad

In [ ]:
# Original range (km) and zimuth (rad) of the data
rr = 1.0e-3 * sweep['range']
aa = sweep['azimuth'] * (np.pi / 180.0)

# Plot using chart.Chart()
_ = chart.Chart(aa, rr, z, style='Z')
_ = chart.Chart(aa, rr, zc, style='Z', title='Corrected Reflectivity (dBZ)')